**TL;DR:**

Исходная задача: преобразовать неструктурированный список должностей в список профессий. Решение строится на получении эмбеддингов должностей с помощью обученной нейронной сети, затем применяется кластеризация методом DBSCAN, так как точное количество кластеров заранее неизвестно и этот метод хорошо работает с неравномерными данными.

Первичный результат был хорошим, но для улучшения предложены следующие шаги:

1. Итеративный DBSCAN на усредненных векторах найленных классов и шумовых данных.
2. Использование метода ближайших соседей для итеративного поиска ближайшего соседа для каждого элемента из класса шума.
3. При необходимости, тщательный подбор параметров первичной кластеризации или использование другой модели для получения эмбеддингов.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from sentence_transformers import SentenceTransformer

import os
import pickle

/Users/timasobornov/Docs/salary/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

rubert_model = SentenceTransformer('cointegrated/rubert-tiny2') 

/Users/timasobornov/Docs/salary/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
def get_rubert_vector(word):
    return rubert_model.encode(word)

def get_distance(vector_1, vector_2, return_distance=False):
    distance = np.linalg.norm(vector_1 - vector_2)
    print("Евклидово расстояние:", distance)

    if return_distance:
        return distance

In [4]:
job_list = pd.read_csv('data/JOB_LIST.csv')
job_list.dropna(inplace=True)
job_list['job_title'] = job_list['job_title'].str.lower()
job_list.head(20)

,job_title
0,"старший инспектор отдела пособий,субсидий и ко..."
1,инспектор отдела муниципальных закупок и имуще...
2,ведущий геолог
3,геолог 1 категории
4,ведущий геолог
5,начальник горного отряда
6,геолог горного отряда
7,"ведущий геолог камеральной группы, старший гео..."
8,"ведущий геолог камеральной группы, главный гео..."
9,"заместитель начальника отдела экономики, промы..."


In [5]:
job_list['job_title'].value_counts(sort=True)[:20]

job_title
бухгалтер               271262
продавец                194165
водитель                156965
продавец-консультант    150922
менеджер                141572
администратор           134027
продавец-кассир         122217
менеджер по продажам    116252
главный бухгалтер        98777
повар                    88128
специалист               82353
кладовщик                78816
кассир                   78761
директор                 76748
воспитатель              75201
подсобный рабочий        66662
юрисконсульт             64882
охранник                 63509
экономист                59172
инженер                  55960
Name: count, dtype: int64

In [6]:
unique_job_list = pd.unique(job_list['job_title'])
unique_job_list = pd.DataFrame(unique_job_list, columns=['job_title'])

In [7]:
# На более мощной машине отдельно просчитали векторы. 
# Если этого не было, то считаем их в рилтайме. 

if os.path.exists('data/vectors.pkl'):
    with open('data/vectors.pkl', 'rb') as f:
        vectors = pickle.load(f)
else:
    vectors = []
    for _, row in unique_job_list.iterrows():

        vector = get_rubert_vector(row['job_title'])
        
        vectors.append(vector)

In [18]:
# Для экомномии времени выполнения, демонстрируем работу алгоритма на части исходных данных, а не полном датасете
sample_size = 500000

vectors_small = vectors[:sample_size]
unique_job_list_small = unique_job_list.copy()
unique_job_list_small = unique_job_list_small[:sample_size]
unique_job_list_small['vector_0'] = vectors_small

In [19]:
dbscan = DBSCAN(eps=0.3, min_samples=3)
dbscan.fit(vectors_small)

unique_job_list_small['class_0'] = dbscan.labels_

In [20]:
classes, counts = np.unique(dbscan.labels_, return_counts=True)

for i, j in zip(classes, counts):
    print(f'Class: {i}, count {j}')

Class: -1, count 218776
Class: 0, count 219167
Class: 1, count 90
Class: 2, count 12
Class: 3, count 6
Class: 4, count 5
Class: 5, count 8
Class: 6, count 20
Class: 7, count 17
Class: 8, count 24
Class: 9, count 3
Class: 10, count 3
Class: 11, count 11
Class: 12, count 14
Class: 13, count 4
Class: 14, count 59
Class: 15, count 12
Class: 16, count 18
Class: 17, count 87
Class: 18, count 3
Class: 19, count 39
Class: 20, count 15
Class: 21, count 14
Class: 22, count 96
Class: 23, count 17
Class: 24, count 14
Class: 25, count 12
Class: 26, count 7
Class: 27, count 9
Class: 28, count 5
Class: 29, count 26
Class: 30, count 36
Class: 31, count 3
Class: 32, count 8
Class: 33, count 3
Class: 34, count 13
Class: 35, count 15
Class: 36, count 8
Class: 37, count 12
Class: 38, count 6
Class: 39, count 9
Class: 40, count 23
Class: 41, count 13
Class: 42, count 56
Class: 43, count 27
Class: 44, count 252
Class: 45, count 165
Class: 46, count 9
Class: 47, count 4
Class: 48, count 4
Class: 49, count 20

In [27]:
unique_job_list_small[unique_job_list_small['class_0'] == 7][:50]

,job_title,vector_0,class_0
51,производитель работ (прораб),"[-0.00851216, -0.01291014, -0.0400405, -0.0178...",7
2001,прораб .(производитель работ ),"[-0.012166404, -0.012252463, -0.04034349, -0.0...",7
12034,прораб(производитель работ),"[-0.009382986, -0.013046221, -0.032301735, -0....",7
14340,производитель работ (прораб) по сварке,"[-0.032245196, -0.027331052, -0.0406869, -0.02...",7
41276,прораб (производитель работ),"[-0.009382986, -0.013046221, -0.032301735, -0....",7
78985,производитель работ (прораб) свик,"[0.00021796564, -0.0156145515, -0.027998785, -...",7
111289,производитель отделочных работ. (прораб),"[0.0033494988, -0.015908912, -0.02792319, -0.0...",7
165520,производитель работ(отделка),"[-0.005338221, -0.02709452, -0.03666384, -0.03...",7
169706,производитель работ(прораб),"[-0.00851216, -0.01291014, -0.0400405, -0.0178...",7
195998,прораб ( производитель работ),"[-0.009382986, -0.013046221, -0.032301735, -0....",7


In [22]:
unique_job_list_small[unique_job_list_small['class_0'] == 20][:100]

,job_title,vector_0,class_0
125,мастер смены,"[0.027568826, -0.02251193, -0.05016806, -0.034...",20
19577,мастер смены тэц,"[0.044571325, -0.0094394265, -0.034007136, -0....",20
56018,"мастер, сменный","[0.031184668, 0.01656102, -0.04844541, -0.0395...",20
64723,мастер смены оф,"[0.017246453, -0.024005458, -0.05929423, -0.02...",20
87091,мастер сменный,"[0.04590152, -0.009157809, -0.032783624, -0.03...",20
115287,мастера смены,"[0.035298042, -0.023109501, -0.06783435, -0.03...",20
159357,мастер смены,"[0.027568826, -0.02251193, -0.05016806, -0.034...",20
174717,мастер сменв,"[0.048574187, -0.016323475, -0.041194662, -0.0...",20
235215,мастер-смены,"[0.03944901, -0.0064592334, -0.04580248, -0.04...",20
253599,мастер смены адс,"[0.034038056, -0.040171333, -0.023878967, -0.0...",20


In [23]:
unique_job_list_small[unique_job_list_small['class_0'] == 21][:100]

,job_title,vector_0,class_0
143,руководитель группы учета,"[0.040974636, 0.004653543, -0.02983334, -0.037...",21
23869,руководитель группы отдела сводного учета,"[0.027811283, 0.0045570666, -0.010662161, -0.0...",21
29452,руководитель расчетной группы,"[0.0041164155, 0.020069756, -0.029324386, -0.0...",21
37807,руководитель сметной группы,"[-0.003914697, 0.014880308, -0.03059929, -0.06...",21
58921,начальник расчетной группы,"[0.021962252, 0.01891547, -0.04474191, -0.0597...",21
62374,руководитель группы сметных расчетов,"[-0.008682183, -0.0013465376, -0.039942026, -0...",21
73588,руководитель группы расчетов,"[0.0031642895, -0.0066291145, -0.04813214, -0....",21
76735,руководитель группы сметного отдела,"[0.006137812, 0.006157382, -0.03319561, -0.062...",21
86344,руководитель группы учета,"[0.040974636, 0.004653543, -0.02983334, -0.037...",21
110190,руководитель группы сметных расчётов,"[-0.016661625, 0.006595547, -0.028222093, -0.0...",21


In [26]:
unique_job_list_small[unique_job_list_small['class_0'] == 24][:100]

,job_title,vector_0,class_0
150,инженер защиты и охраны леса,"[0.044147488, 0.04958962, 0.039636776, -0.0915...",24
15126,инженер по охране и защите леса,"[0.040978927, 0.07233069, 0.022543201, -0.0692...",24
48871,инженер охраны и защиты леса,"[0.043570846, 0.050354, 0.03984548, -0.0895093...",24
75430,инспектор по охране и защите леса,"[0.06491234, 0.056659937, 0.018852146, -0.0738...",24
105547,инспектор по охране леса на участке охраны и ...,"[0.05521, 0.03014421, 0.016973238, -0.08058007...",24
154907,инженер по охране и защите леса,"[0.040978927, 0.07233069, 0.022543201, -0.0692...",24
241966,"инженер по охране, защите леса","[0.009680278, 0.05805455, 0.012218982, -0.0693...",24
266853,инженер по защите и охране леса,"[0.040278174, 0.070944436, 0.024196174, -0.069...",24
267660,иженер охраны и защиты леса,"[0.059490386, 0.040281903, 0.06630536, -0.0956...",24
270976,специалист по охране и защите леса,"[0.068502575, 0.056194585, 0.020226233, -0.067...",24


## Доуточнение с помощью итеративного DBSCAN

In [28]:
# Вычисляем средние значения 'vector_0' для каждого класса, кроме классов -1 и 0
mean_vectors = unique_job_list_small[unique_job_list_small['class_0'].isin([-1, 0]) == False].groupby('class_0')['vector_0'].transform('mean')

# Создаем новую колонку 'mean_vector_0', где:
# - для классов -1 и 0 сохраняем исходное значение 'vector_0'
# - для остальных классов присваиваем среднее значение 'vector_0' внутри соответствующего класса
unique_job_list_small['mean_vector_0'] = unique_job_list_small.apply(
    lambda row: row['vector_0'] if row['class_0'] in [-1, 0]  # если класс -1 или 0, берем исходный 'vector_0'
    else mean_vectors[row.name],  # иначе берем среднее значение для данного класса
    axis=1  # применяем функцию к каждой строке
)

In [29]:
# Выбираем все строки, где значение в колонке 'class_0' равно -1 или 0
df_negative_or_zero = unique_job_list_small[unique_job_list_small['class_0'].isin([-1, 0])]

# Выбираем по одной случайной строке для каждого класса, кроме классов -1 и 0
df_other_classes = unique_job_list_small[unique_job_list_small['class_0'].isin([-1, 0]) == False].groupby('class_0').sample(n=1, random_state=1)

# Объединяем два DataFrame: все строки с классами -1 и 0 + по одной строке из остальных классов
unique_job_list_small = pd.concat([df_negative_or_zero, df_other_classes])

unique_job_list_small.head(20)

,job_title,vector_0,class_0,mean_vector_0
0,"старший инспектор отдела пособий,субсидий и ко...","[0.030501965, 0.012508502, -0.055463985, -0.13...",-1,"[0.030501965, 0.012508502, -0.055463985, -0.13..."
1,инспектор отдела муниципальных закупок и имуще...,"[0.11537887, -0.001824434, 0.030480817, -0.089...",-1,"[0.11537887, -0.001824434, 0.030480817, -0.089..."
2,ведущий геолог,"[-0.02707592, 0.033282503, 0.03790604, -0.1112...",0,"[-0.02707592, 0.033282503, 0.03790604, -0.1112..."
4,начальник горного отряда,"[0.031840302, 0.0015570363, -0.008545109, -0.1...",-1,"[0.031840302, 0.0015570363, -0.008545109, -0.1..."
5,геолог горного отряда,"[0.032704916, 0.010802099, 0.027436346, -0.100...",-1,"[0.032704916, 0.010802099, 0.027436346, -0.100..."
6,"ведущий геолог камеральной группы, старший гео...","[0.012428342, 0.03692716, 0.039867546, -0.0982...",-1,"[0.012428342, 0.03692716, 0.039867546, -0.0982..."
7,"ведущий геолог камеральной группы, главный гео...","[-0.048368137, 0.025798205, 0.017690176, -0.11...",0,"[-0.048368137, 0.025798205, 0.017690176, -0.11..."
8,"заместитель начальника отдела экономики, промы...","[0.08575809, -0.025911864, -0.0017935511, -0.1...",-1,"[0.08575809, -0.025911864, -0.0017935511, -0.1..."
9,"заместитель генерального директора, начальник ...","[-0.0003442502, -0.023291621, -0.038129363, -0...",0,"[-0.0003442502, -0.023291621, -0.038129363, -0..."
10,заместитель генерального директора,"[0.0014275342, -0.036567856, -0.034488212, -0....",0,"[0.0014275342, -0.036567856, -0.034488212, -0...."


In [30]:
# Обучаем DBSCAN

dbscan = DBSCAN(eps=0.3, min_samples=3)
dbscan.fit(list(unique_job_list_small['mean_vector_0']))
unique_job_list_small['class_1'] = dbscan.labels_

In [31]:
classes, counts = np.unique(dbscan.labels_, return_counts=True)

for i, j in zip(classes, counts):
    print(f'Class: {i}, count {j}')

Class: -1, count 226930
Class: 0, count 219815
Class: 1, count 6
Class: 2, count 3
Class: 3, count 3
Class: 4, count 4
Class: 5, count 3
Class: 6, count 3
Class: 7, count 3
Class: 8, count 5
Class: 9, count 3
Class: 10, count 4
Class: 11, count 3
Class: 12, count 4
Class: 13, count 3
Class: 14, count 4
Class: 15, count 3
Class: 16, count 4
Class: 17, count 4
Class: 18, count 3
Class: 19, count 4
Class: 20, count 3
Class: 21, count 3
Class: 22, count 5
Class: 23, count 4
Class: 24, count 3
Class: 25, count 3
Class: 26, count 4
Class: 27, count 3
Class: 28, count 3
Class: 29, count 3
Class: 30, count 4
Class: 31, count 3
Class: 32, count 3
Class: 33, count 3
Class: 34, count 3
Class: 35, count 3
Class: 36, count 4
Class: 37, count 3
Class: 38, count 3
Class: 39, count 3
Class: 40, count 3
Class: 41, count 4
Class: 42, count 3
Class: 43, count 3
Class: 44, count 3
Class: 45, count 3
Class: 46, count 5
Class: 47, count 3
Class: 48, count 3
Class: 49, count 3
Class: 50, count 3
Class: 51, c

In [32]:
unique_job_list_small[unique_job_list_small['class_1'] == 21][:100]

,job_title,vector_0,class_0,mean_vector_0,class_1
53836,организатор досуга,"[0.06817025, -0.0114520425, 0.047077913, -0.08...",-1,"[0.06817025, -0.0114520425, 0.047077913, -0.08...",21
469312,организатор досуга подопечных,"[0.07236333, -0.01623024, 0.022799172, -0.0966...",-1,"[0.07236333, -0.01623024, 0.022799172, -0.0966...",21
266678,организатор досуговой работы,"[0.059032414, -0.005196988, 0.036127422, -0.08...",5712,"[0.059970878064632416, -0.0019067727262154222,...",21


In [33]:
unique_job_list_small[unique_job_list_small['class_1'] == 22][:100]

,job_title,vector_0,class_0,mean_vector_0,class_1
60737,"медицинская сестра диагностического отделения,...","[-0.04630151, -0.013159464, 0.01761111, -0.144...",-1,"[-0.04630151, -0.013159464, 0.01761111, -0.144...",22
68154,медицинская сестра участковая; медицинская сес...,"[-0.051428318, -0.016395112, 0.03557915, -0.20...",-1,"[-0.051428318, -0.016395112, 0.03557915, -0.20...",22
153328,"старшая медицинская сестра, участковая медицин...","[-0.053858664, -0.019494811, -0.029806005, -0....",-1,"[-0.053858664, -0.019494811, -0.029806005, -0....",22
192367,"медицинская сестра опер. блока, медицинская се...","[-0.05179934, 0.0018882231, 0.0048510022, -0.1...",-1,"[-0.05179934, 0.0018882231, 0.0048510022, -0.1...",22
314519,"участковая медицинская сестра, процедурная мед...","[-0.06825189, -0.026145194, 0.006993107, -0.17...",5989,"[-0.05233485500017802, -0.0241134117046992, -0...",22


In [34]:
unique_job_list_small[unique_job_list_small['class_1'] == 23][:100]

,job_title,vector_0,class_0,mean_vector_0,class_1
61044,специалист отдела возвратов,"[-0.0035018262, -0.024023663, -0.007412191, -0...",-1,"[-0.0035018262, -0.024023663, -0.007412191, -0...",23
324913,специалист по работе с возвратами,"[-0.0037238328, -0.004878256, -0.012685444, -0...",-1,"[-0.0037238328, -0.004878256, -0.012685444, -0...",23
356985,специалист отдела возврата,"[-0.007829693, -0.02585615, -0.013646189, -0.0...",-1,"[-0.007829693, -0.02585615, -0.013646189, -0.0...",23
197054,специалист по обработке возврата,"[-0.01358591, -0.029933516, -0.004568601, -0.0...",7353,"[-0.012316608801484108, -0.022568408399820328,...",23


## Доуточнение с помощью NearestNeighbors

In [35]:
no_noize_vectors = list(unique_job_list_small[unique_job_list_small['class_0'].isin([-1, 0]) == False]['vector_0'])
no_noize_classes = list(unique_job_list_small[unique_job_list_small['class_0'].isin([-1, 0]) == False]['class_0'])

knn = NearestNeighbors(n_neighbors=1)
knn.fit(no_noize_vectors)

NearestNeighbors(n_neighbors=1)

In [36]:
class_1 = {}

for index, row in unique_job_list_small[unique_job_list_small['class_0'].isin([-1, 0])].iterrows():
    distances, indices = knn.kneighbors(row['vector_0'].reshape(1, -1), n_neighbors=1)
    class_1[index] = no_noize_classes[indices.item()]

unique_job_list_small['class_1'] = unique_job_list_small['class_0']

for key, value in class_1.items():
    unique_job_list_small.loc[key, 'class_1'] = value

In [37]:
unique_job_list_small.head(20)

,job_title,vector_0,class_0,mean_vector_0,class_1
0,"старший инспектор отдела пособий,субсидий и ко...","[0.030501965, 0.012508502, -0.055463985, -0.13...",-1,"[0.030501965, 0.012508502, -0.055463985, -0.13...",6679
1,инспектор отдела муниципальных закупок и имуще...,"[0.11537887, -0.001824434, 0.030480817, -0.089...",-1,"[0.11537887, -0.001824434, 0.030480817, -0.089...",1898
2,ведущий геолог,"[-0.02707592, 0.033282503, 0.03790604, -0.1112...",0,"[-0.02707592, 0.033282503, 0.03790604, -0.1112...",128
4,начальник горного отряда,"[0.031840302, 0.0015570363, -0.008545109, -0.1...",-1,"[0.031840302, 0.0015570363, -0.008545109, -0.1...",9367
5,геолог горного отряда,"[0.032704916, 0.010802099, 0.027436346, -0.100...",-1,"[0.032704916, 0.010802099, 0.027436346, -0.100...",128
6,"ведущий геолог камеральной группы, старший гео...","[0.012428342, 0.03692716, 0.039867546, -0.0982...",-1,"[0.012428342, 0.03692716, 0.039867546, -0.0982...",6425
7,"ведущий геолог камеральной группы, главный гео...","[-0.048368137, 0.025798205, 0.017690176, -0.11...",0,"[-0.048368137, 0.025798205, 0.017690176, -0.11...",7286
8,"заместитель начальника отдела экономики, промы...","[0.08575809, -0.025911864, -0.0017935511, -0.1...",-1,"[0.08575809, -0.025911864, -0.0017935511, -0.1...",8529
9,"заместитель генерального директора, начальник ...","[-0.0003442502, -0.023291621, -0.038129363, -0...",0,"[-0.0003442502, -0.023291621, -0.038129363, -0...",7025
10,заместитель генерального директора,"[0.0014275342, -0.036567856, -0.034488212, -0....",0,"[0.0014275342, -0.036567856, -0.034488212, -0....",1349


In [49]:
unique_job_list_small[unique_job_list_small['class_1'] == 1][:50]

,job_title,vector_0,class_0,mean_vector_0,class_1
1473,инженер-топограф,"[-0.010013919, 0.036876537, 0.0044496716, -0.0...",0,"[-0.010013919, 0.036876537, 0.0044496716, -0.0...",1
4916,инженер-картограф,"[-0.019532913, 0.028782489, 0.0030482863, -0.0...",0,"[-0.019532913, 0.028782489, 0.0030482863, -0.0...",1
19254,техник-топограф,"[0.010134748, 0.01967238, 0.016243173, -0.0823...",0,"[0.010134748, 0.01967238, 0.016243173, -0.0823...",1
100860,топограф-геодезист,"[-0.008572009, 0.03146165, 0.026152084, -0.096...",0,"[-0.008572009, 0.03146165, 0.026152084, -0.096...",1
168320,инженер-картограф ii кат,"[-0.017138889, 0.03375821, 0.027736485, -0.085...",0,"[-0.017138889, 0.03375821, 0.027736485, -0.085...",1
168321,инженер-картограф i кат,"[-0.033911332, 0.04241836, 0.024479907, -0.092...",0,"[-0.033911332, 0.04241836, 0.024479907, -0.092...",1
177916,инженер топограф,"[-0.008580241, 0.016277783, -0.004847866, -0.0...",0,"[-0.008580241, 0.016277783, -0.004847866, -0.0...",1
182033,"геофизик 1кат, ответственный исполнитель","[-0.052311644, 0.04623198, 0.010356098, -0.115...",-1,"[-0.052311644, 0.04623198, 0.010356098, -0.115...",1
187788,геодезист-топограф,"[-0.016078653, 0.027825283, 0.039463747, -0.10...",0,"[-0.016078653, 0.027825283, 0.039463747, -0.10...",1
196996,инженер - топограф,"[-0.010013919, 0.036876537, 0.0044496716, -0.0...",0,"[-0.010013919, 0.036876537, 0.0044496716, -0.0...",1


In [50]:
unique_job_list_small[unique_job_list_small['class_1'] == 2][:50]

,job_title,vector_0,class_0,mean_vector_0,class_1
1486,слесарь по ремонту оборудования - 5 разряда,"[-0.054416765, -0.0015305813, -0.0084739635, -...",0,"[-0.054416765, -0.0015305813, -0.0084739635, -...",2
2473,слесарь по ремонту технологических установок 5...,"[-0.05578018, -0.02521428, -0.010282361, -0.06...",0,"[-0.05578018, -0.02521428, -0.010282361, -0.06...",2
5553,слесарь-ремонтник 5 разряда,"[-0.039965328, 0.0067225904, -0.02988637, -0.0...",0,"[-0.039965328, 0.0067225904, -0.02988637, -0.0...",2
18996,слесарь-ремонтник 5-го разряда,"[-0.031951353, 0.00082882855, -0.016506974, -0...",0,"[-0.031951353, 0.00082882855, -0.016506974, -0...",2
26515,слесарь ремонтник металлургического оборудован...,"[-0.04190941, -0.037415214, 0.0015950393, -0.0...",0,"[-0.04190941, -0.037415214, 0.0015950393, -0.0...",2
35758,слесарь ремонтник 5-го разряда,"[-0.034159254, -0.018567085, -0.016573304, -0....",0,"[-0.034159254, -0.018567085, -0.016573304, -0....",2
35801,слесарь -ремонтник 5 разряда,"[-0.039965328, 0.0067225904, -0.02988637, -0.0...",0,"[-0.039965328, 0.0067225904, -0.02988637, -0.0...",2
40539,ремонтно вспомогательная служба. слесарь ремон...,"[-0.044093683, -0.001988537, -0.021102475, -0....",0,"[-0.044093683, -0.001988537, -0.021102475, -0....",2
45144,слесарь по ремонту ру 5 разряда,"[-0.04103485, -0.008484924, -0.0257925, -0.027...",0,"[-0.04103485, -0.008484924, -0.0257925, -0.027...",2
45327,слесарь по ремонту обогатительного оборудовани...,"[-0.037709087, -0.0041465983, 0.025835618, -0....",0,"[-0.037709087, -0.0041465983, 0.025835618, -0....",2


In [51]:
unique_job_list_small[unique_job_list_small['class_1'] == 3][:50]

,job_title,vector_0,class_0,mean_vector_0,class_1
52652,и.о. визуального мерчандайзера,"[-0.03473378, -0.0028455246, 0.08226677, -0.10...",-1,"[-0.03473378, -0.0028455246, 0.08226677, -0.10...",3
72825,визуализатор,"[-0.011128927, 0.008086954, 0.061336674, -0.06...",-1,"[-0.011128927, 0.008086954, 0.061336674, -0.06...",3
80488,визитный мерчендайзер,"[-0.028782245, 0.002719583, 0.034471825, -0.07...",0,"[-0.028782245, 0.002719583, 0.034471825, -0.07...",3
200116,концептуальный мерчендайзер,"[-0.049790733, 0.005553606, 0.09275365, -0.100...",-1,"[-0.049790733, 0.005553606, 0.09275365, -0.100...",3
218033,визитный мерчандайзер,"[-0.019632729, 0.0034965684, 0.028283576, -0.0...",0,"[-0.019632729, 0.0034965684, 0.028283576, -0.0...",3
236616,мерчендайзера,"[-0.029471878, -0.022754109, 0.021784969, -0.0...",0,"[-0.029471878, -0.022754109, 0.021784969, -0.0...",3
292530,маршрутный мерчендайзер,"[-0.05344708, 0.003651655, 0.043224193, -0.082...",0,"[-0.05344708, 0.003651655, 0.043224193, -0.082...",3
295114,логист мерчендайзер,"[-0.07528735, -0.000559254, 0.060592387, -0.09...",0,"[-0.07528735, -0.000559254, 0.060592387, -0.09...",3
359141,приходящий мерчендайзер,"[-0.048810635, -0.015926566, 0.05332703, -0.07...",0,"[-0.048810635, -0.015926566, 0.05332703, -0.07...",3
415938,пеший мерчендайзер,"[-0.031728964, 0.0010302605, 0.046338484, -0.0...",0,"[-0.031728964, 0.0010302605, 0.046338484, -0.0...",3


## Если исходный результат не устраивает, стоит обратить внимание на другие модели для извлечения признаков или более тщательно подбирать параметры класстеризации